In [ ]:
import pandas as pd
import altair as alt
import altair_viewer

################################################
import tkinter as tk
from tkinter import filedialog

window = tk.Tk()
window.title('LPP Anywhere App')
window.geometry("350x170")

def getCSV ():
    global data
    import_file_path = filedialog.askopenfilename()
    data = pd.read_csv (import_file_path)
    
def getentry():
        global name
        #print (entry.get()),
        name = entry.get(),
        print(type(name))
        window.destroy()

labelmain = tk.Label(
            text = '''LPP_Anywhere
                1. Enter Part Type
                2. Please upload the data as per the template
                3. Press Submit''',
            background = "#34A2FE",
            foreground = "white",
            width = 50,
            height = 5
)

partentry = tk.Label(
                text = 'Enter Part Type:',
                fg = 'black'
)

btn = tk.Button(master = window,
        text = 'Submit',
        width = 10,
        height = 1,
        fg = 'white',
        bg = 'black',
        font = ('helvetica', 8, 'bold'),
        command = getentry,
        relief = tk.RAISED
)

entry = tk.Entry()

labelfooter = tk.Label(
                text = 'Developed by Abhinav Srivastava & Abhay Ingale',
                fg = 'black',
                font = ('helvetica', 6)
)

#bckg = PhotoImage(file = 'E:\\JS\\Py\\Tata.jpg')
#labelback = tk.Label(window, image = bckg)
#labelback.place(x=0, y=0, relwidth=1, relheight=1)

browseButton_CSV = tk.Button(text="Upload Template", command=getCSV, bg='green', fg='white', font=('helvetica', 8, 'bold'))

#labelback.grid(row=0, rowspan = 2, columnspan = 2)
labelmain.grid(row=0,columnspan=2)
partentry.grid(row=1,column=0)
entry.grid(row=1,column=1)
browseButton_CSV.grid(row=2,column=0)
btn.grid(row=2,column=1)
labelfooter.grid(row=3,column=0)


window.mainloop()
###############################################

#data = pd.read_csv("E:/JS/Py/LPPTest.csv", sep=',')
column_names = data.head(0)
data.columns = ['OEM','Model','PN','CD','PD','PO']
data.PN = data.PN.astype('str')
#create selection and bind with legend (not working for layered charts)
selection = alt.selection_multi(fields=['OEM'], bind='legend')


#create plots
plot1 = alt.Chart(data).mark_point(filled=True, opacity = 1).encode(
    alt.Y('PO',
          scale = alt.Scale(domain=(data.PO.min()*0.95,data.PO.max()*1.05)), title = column_names.columns[5]),
    alt.X('CD',
          scale = alt.Scale(domain=(data.CD.min()*0.95,data.CD.max()*1.05)), title = column_names.columns[3]),
    color = 'OEM',
    tooltip = ['OEM','Model','PN','CD','PO'],
    #opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
).properties(
    title = 'Price vs. Cost Driver of '+name[0]
)
plot1.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

plot2 = alt.Chart(data).mark_point(filled=True, opacity = 1).encode(
    alt.Y('CD',
          scale = alt.Scale(domain=(data.CD.min()*0.95,data.CD.max()*1.05)), title = column_names.columns[3]),
    alt.X('PD',
          scale = alt.Scale(domain=(data.PD.min()*0.95,data.PD.max()*1.05)), title = column_names.columns[4]),
    color = 'OEM',
    tooltip = ['OEM','Model','PN','CD','PD']
).properties(
    title = 'Cost Driver vs. Performance Driver of '+name[0]
)

plot3 = alt.Chart(data).mark_point(filled=True, opacity = 1).encode(
    alt.Y('PO',
          scale = alt.Scale(domain=(data.PO.min()*0.95,data.PO.max()*1.05)), title = column_names.columns[5]),
    alt.X('PD',
          scale = alt.Scale(domain=(data.PD.min()*0.95,data.PD.max()*1.05)), title = column_names.columns[4]),
    color = 'OEM',
    tooltip = ['OEM','Model','PN','PD','PO']
).properties(
    title = 'Price vs. Performance Driver of '+name[0]
)

#create labels
text1 = plot1.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='PN'
)

text2 = plot2.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='PN'
)

text3 = plot3.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='PN'
)

#create regression lines
reg1 = plot1.transform_regression('CD','PO',method = 'linear'
).mark_line()

reg2 = plot2.transform_regression('PD','CD',method = 'linear'
).mark_line(color='red')

reg3 = plot3.transform_regression('PO','PD',method = 'linear'
).mark_line(color='red')

#calculate r2
r21 = alt.Chart(data).transform_regression(
    'CD', 'PO', params=True
).mark_text(align='left').encode(
    x=alt.value(20),  # pixels from left
    y=alt.value(20),  # pixels from top
    text= 'rSquared:N' #'coef:O'
)

r22 = alt.Chart(data).transform_regression(
    'PD', 'CD', params=True
).mark_text(align='left').encode(
    x=alt.value(20),  # pixels from left
    y=alt.value(20),  # pixels from top
    text= 'rSquared:N' #'coef:O'
)

r23 = alt.Chart(data).transform_regression(
    'PO', 'PD', params=True
).mark_text(align='left').encode(
    x=alt.value(20),  # pixels from left
    y=alt.value(20),  # pixels from top
    text= 'rSquared:N' #'coef:O'
)

boxp_PO = alt.Chart(data).mark_boxplot().encode(
    alt.Y('PO',
          scale = alt.Scale(domain=(data.PO.min()*0.95,data.PO.max()*1.05)), 
          title = column_names.columns[5]),
          #axis = alt.Axis(labels = False),
          #labels = None
)

boxp_CD = alt.Chart(data).mark_boxplot().encode(
    alt.Y('CD',
          scale = alt.Scale(domain=(data.CD.min()*0.95,data.CD.max()*1.05)), title = column_names.columns[3])
)

boxp_PD = alt.Chart(data).mark_boxplot().encode(
    alt.X('PD',
          scale = alt.Scale(domain=(data.PD.min()*0.95,data.PD.max()*1.05)), title = column_names.columns[4])
)

boxset = alt.hconcat(boxp_PO, boxp_CD, boxp_PD)

print(column_names)

#layer, concat and display interactive
chart = alt.hconcat((plot1+reg1+r21).interactive(), boxp_PO, (plot2+reg2+r22).interactive(), boxp_CD)
chart1 = alt.vconcat((plot3+reg3+r23).interactive(), boxp_PD)
chart2 = alt.vconcat(chart, chart1)
#chart2
chart2.save('LPP for '+name[0]+'.html', embed_options={'renderer':'svg'})
chart2.show()

#chart = alt.hconcat((plot1+reg1+r21).interactive(), (plot2+reg2+r22).interactive())
#chart1 = alt.hconcat((plot3+reg3+r23).interactive())
#alt.vconcat(chart, chart1)


# to print with data labels and single row
#chart = alt.hconcat(plot1+text1+reg1, plot2+text2+reg2, plot3+text3+reg3)
#chart